In [ ]:
print("Hello World!")

Hello World!


In [2]:
#!pip install openai python-dotenv wikipedia

In [ ]:
import os
import openai

# Retrieve your API key from Codespaces environment
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("No OpenAI API key found! Make sure OPENAI_API_KEY is set as a Codespaces secret.")

openai.api_key = api_key

In [6]:
import wikipedia

# --- Tools ---
def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def wiki_search(query: str) -> str:
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Error: {e}"

tools = {
    "calculator": calculator,
    "wiki": wiki_search,
}

# --- Agent Loop ---
def run_agent(query: str, max_steps=3):
    print(f"User: {query}\n")

    context = """You are an intelligent agent that can reason step by step.
    You have access to these tools:
    - calculator(expression): evaluate math expressions and return a number
    - wiki(query): return a short summary of the topic

    You will follow this reasoning format strictly:

    Thought: describe your reasoning
    Action: choose either calculator(), wiki(), or ANSWER()
    Observation: result of the previous action

    Important:
    - Use at most ONE tool per step.
    - If you already have the information you need, respond with:
    Action: ANSWER(<final answer here>)
    - Do NOT repeat the same tool call multiple times.
    - Stop once you have provided the final answer.
    """

    for step in range(max_steps):
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": f"User question: {query}\nPrevious steps:\n{context}"}
            ]
        )
        text = response.choices[0].message.content
        print(text)

        # Parse the action
        lines = text.strip().splitlines()
        action_line = [l for l in lines if l.startswith("Action:")]
        if not action_line:
            break

        action = action_line[-1].replace("Action:", "").strip()

        if "ANSWER" in action:
            final = action.split("ANSWER")[-1].strip(" :()")
            print(f"\nFinal Answer: {final}")
            return  # instead of break, to stop before next Observation print

        else:
            # Expect "tool_name(input)"
            if "(" in action and action.endswith(")"):
                tool_name = action.split("(")[0]
                tool_input = action[len(tool_name)+1:-1]

                if tool_name in tools:
                    observation = tools[tool_name](tool_input)
                else:
                    observation = f"Unknown tool: {tool_name}"

                #context += f"{text}\nObservation: {observation}\n"
                print(f"Observation: {observation}\n")


In [9]:
run_agent("What is 23 * 7?")

User: What is 23 * 7?



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <your-ap*******ere>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
run_agent("Who is Marie Curie?")

User: Who is Marie Curie?



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************MYQA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}